In [1]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mai-ml-hw-models/Train.csv
/kaggle/input/mai-ml-hw-models/Test.csv
/kaggle/input/mai-ml-hw-models/SampleSubmission.csv


In [2]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb 
import xgboost as xgb 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler

In [3]:
def sign(x):
    if x>0:
        return 1
    elif x< 0:
        return -1
    else:
        return 0

In [4]:
read_data = pd.read_csv('/kaggle/input/mai-ml-hw-models/Train.csv')
read_data.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,0,0,0,0,0,0,18473000
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,4157000
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,0,0,0,0,0,0,4530000
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,0,0,0,0,0,0,7981000
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,0,0,0,0,0,0,11888000


In [5]:
# первая модель
read_data_1 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Train.csv')
X_1 = read_data_1.drop(['date', 'id', 'price'],axis = 1)
y_1 = read_data_1.price
xgb_reg_1 = xgb.XGBRegressor(n_estimators=2000, learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7)
xgb_reg_1.fit(X_1,y_1)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[17:53:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.75, verbosity=1)

In [6]:
def new_fitch_2(data,read_data):
    max_rooms = read_data.rooms.max()
    min_rooms = read_data.rooms.min()
    max_n_photots = read_data.n_photos.max()
    min_n_photos = read_data.n_photos.min()
    max_metro_dist = read_data.metro_dist.max()
    min_metro_dist = read_data.metro_dist.min()
    size_col = read_data.shape[0]
    nan_metro_dist = int(np.mean(read_data.metro_dist))
    nan_build_tech = 1
    nan_g_lift = 1
    data.build_tech[read_data.build_tech.isna()] = nan_build_tech
    data.metro_dist[read_data.metro_dist.isna()] = nan_metro_dist
    data.g_lift[read_data.g_lift.isna()] = nan_g_lift
    data['area_n_photos'] =data['area']*data['n_photos']
    data['area_n_photos'] =data['area_n_photos'].apply(sign)
    data['area_metro_dist'] = data.area*data.metro_dist
    data['area_metro_dist'] = (data['area_metro_dist'] - np.mean(data['area_metro_dist']))/np.std(data['area_metro_dist'])
    data['area_balcon'] = (data.area*data.balcon).apply(sign)
    data['fast_down'] = (read_data.metro_dist*read_data.g_lift).apply(sign)
    data['flor_metro_dist'] = (data.metro_dist + data.floor)
# area
#     цена комнаты в райное
    tmp = read_data.groupby('area',as_index=False).sum()[['area','rooms','price']]
    price_room = pd.DataFrame({'area': tmp.area, 'price_room_area':tmp.price/tmp.rooms } ,index = np.arange(0,tmp.price.shape[0])) 
    data['price_room_area'] = 0
    for index,row in price_room.iterrows():
        data.price_room_area[data.area == row.area] = row.price_room_area
    data.price_room_area = (data.price_room_area - data.price_room_area.mean())/data.price_room_area.std()
# средняя цена комнаты
    data['mean_price_room'] = tmp.price.mean()
# дисперсия цены комнаты
    data['std_price_room'] = tmp.price.std()
# max price room
    data['max_price_room'] = tmp.price.max()
# min price room
    data['min_price_room'] = tmp.price.min()

#     количесво фоток района
    n_photos_area = read_data.groupby('area',as_index=False).sum()[['area','n_photos']]
    data['n_photos_area'] = 0
    for index,row in n_photos_area.iterrows():
        data.n_photos_area[data.area == row.area] = row.n_photos
# средняя количество фоток
    data['mean_n_photos_area.n_photos'] = n_photos_area.n_photos.mean()
# дисперсия количество фоток
    data['std_n_photos_area.n_photos'] = n_photos_area.n_photos.std()
# max количество фоток
    data['max_n_photos_area.n_photos'] = n_photos_area.n_photos.max()
# min количество фоток
    data['min_n_photos_area.n_photos'] = n_photos_area.n_photos.min()

#     среднее растояние метро
    metro_dist_area = read_data.groupby('area',as_index=False).mean()[['area','metro_dist']]
    data['metro_dist_area'] = 0
    for index,row in metro_dist_area.iterrows():
        data.metro_dist_area[data.area == row.area] = row.metro_dist
# средняя metro_dist
    data['mean_n_photos_area.metro_dist'] = metro_dist_area.metro_dist.mean()
# дисперсия metro_dist
    data['std_n_photos_area.metro_dist'] = metro_dist_area.metro_dist.std()
# max metro_dist
    data['max_n_photos_area.metro_dist'] = metro_dist_area.metro_dist.max()
# min metro_dist
    data['min_n_photos_area.metro_dist'] = metro_dist_area.metro_dist.min()
# street
    tmp = read_data.groupby('street_id',as_index=False).sum()[['street_id','rooms','price']]
    price_room_street = pd.DataFrame({'street_id': tmp.street_id, 'price_room_street':tmp.price/tmp.rooms } ,index = np.arange(0,tmp.price.shape[0])) 
    data['price_room_street'] = 0
    for index,row in price_room_street.iterrows():
        data.price_room_street[data.street_id == row.street_id] = row.price_room_street
    data.price_room_street = (data.price_room_street - data.price_room_street.mean())/data.price_room_street.std()
# средняя цена комнаты
    data['mean_price_room_street'] = tmp.price.mean()
# дисперсия цены комнаты
    data['std_price_room_street'] = tmp.price.std()
# max price room
    data['max_price_room_street'] = tmp.price.max()
# min price room
    data['min_price_room_street'] = tmp.price.min()

# количесво фоток района
    n_photos_street = read_data.groupby('street_id',as_index=False).sum()[['street_id','n_photos']]
    data['n_photos_street'] = 0
    for index,row in n_photos_street.iterrows():
        data.n_photos_street[data.street_id == row.street_id] = row.n_photos
# средняя количество фоток
    data['mean_n_photos_street'] = n_photos_street.n_photos.mean()
# дисперсия количество фоток
    data['std_n_photos_street'] = n_photos_street.n_photos.std()
# max количество фоток
    data['max_n_photos_street'] = n_photos_street.n_photos.max()
# min количество фоток
    data['min_n_photos_street'] = n_photos_street.n_photos.min()

#     среднее растояние метро
    metro_dist_street = read_data.groupby('street_id',as_index=False).mean()[['street_id','metro_dist']]
    data['metro_dist_street'] = 0
    for index,row in metro_dist_street.iterrows():
        data.metro_dist_street[data.street_id == row.street_id] = row.metro_dist  
        
# средняя metro_dist
    data['mean_n_photos_street.metro_dist'] = metro_dist_street.metro_dist.mean()
# дисперсия metro_dist
    data['std_n_photos_street.metro_dist'] = metro_dist_street.metro_dist.std()
# max metro_dist
    data['max_n_photos_street.metro_dist'] = metro_dist_street.metro_dist.max()
# min metro_dist
    data['min_n_photos_street.metro_dist'] = metro_dist_street.metro_dist.min()    

In [7]:
parameters_xgb_reg = {
    'gamma': 0,
    'colsample_bytree': 0.7, 
    'learning_rate': 0.07,
    'max_depth': 7, 
    'min_child_weight': 3,  
    'nthread': 4, 
    'objective': 'reg:linear', 
    'random_state': 42, 
    'silent': 1, 
    'subsample': 0.7,
    "n_estimators": 500,
}

In [8]:
# вторая модель
read_data_2 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Train.csv')
new_fitch_2(read_data_2,read_data)
X_2 = read_data_2.drop(['date', 'id', 'price'],axis = 1)
y_2 = read_data_2.price
xgb_reg_2 = xgb.XGBRegressor(**parameters_xgb_reg)
xgb_reg_2.fit(X_2,y_2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: Setti

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=3, missing=None, n_estimators=500,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [9]:
def new_fitch_3(data,read_data):
    max_rooms = read_data.rooms.max()
    min_rooms = read_data.rooms.min()
    max_n_photots = read_data.n_photos.max()
    min_n_photos = read_data.n_photos.min()
    max_metro_dist = read_data.metro_dist.max()
    min_metro_dist = read_data.metro_dist.min()
    size_col = read_data.shape[0]
    nan_metro_dist = int(np.mean(read_data.metro_dist))
    nan_build_tech = 1
    nan_g_lift = 1
    data.build_tech[read_data.build_tech.isna()] = nan_build_tech
    data.metro_dist[read_data.metro_dist.isna()] = nan_metro_dist
    data.g_lift[read_data.g_lift.isna()] = nan_g_lift
    data['area_n_photos'] =data['area']*data['n_photos']
    data['area_n_photos'] =data['area_n_photos'].apply(sign)
    data['area_metro_dist'] = data.area*data.metro_dist
    data['area_metro_dist'] = (data['area_metro_dist'] - np.mean(data['area_metro_dist']))/np.std(data['area_metro_dist'])
    data['area_balcon'] = (data.area*data.balcon).apply(sign)
    data['fast_down'] = (read_data.metro_dist*read_data.g_lift).apply(sign)
    data['flor_metro_dist'] = (data.metro_dist + data.floor)
# area
# фото
# количество
    tmp = read_data.groupby('area',as_index = False).sum()[['area','n_photos']]
    data['sum_photos_area'] = 0
    for index,row in tmp.iterrows():
        data.sum_photos_area[data.area == row.area] = row.n_photos
# mean
    tmp = read_data.groupby('area',as_index = False).mean()[['area','n_photos']]
    data['mean_photos_area'] = 0
    for index,row in tmp.iterrows():
        data.mean_photos_area[data.area == row.area] = row.n_photos
    
# std
    tmp = read_data.groupby('area',as_index = False).std()[['area','n_photos']]
    data['std_photos_area'] = 0
    for index,row in tmp.iterrows():
        data.std_photos_area[data.area == row.area] = row.n_photos
    
# min
    tmp = read_data.groupby('area',as_index = False).min()[['area','n_photos']]
    data['min_photos_area'] = 0
    for index,row in tmp.iterrows():
        data.min_photos_area[data.area == row.area] = row.n_photos  
# max
    tmp = read_data.groupby('area',as_index = False).max()[['area','n_photos']]
    data['max_photos_area'] = 0
    for index,row in tmp.iterrows():
        data.max_photos_area[data.area == row.area] = row.n_photos
# metro_dist
# количество
    tmp = read_data.groupby('area',as_index = False).sum()[['area','metro_dist']]
    data['sum_metro_dist_area'] = 0
    for index,row in tmp.iterrows():
        data.sum_metro_dist_area[data.area == row.area] = row.metro_dist
# mean
    tmp = read_data.groupby('area',as_index = False).mean()[['area','metro_dist']]
    data['mean_metro_dist_area'] = 0
    for index,row in tmp.iterrows():
        data.mean_metro_dist_area[data.area == row.area] = row.metro_dist
    
# std
    tmp = read_data.groupby('area',as_index = False).std()[['area','metro_dist']]
    data['std_metro_dist_area'] = 0
    for index,row in tmp.iterrows():
        data.std_metro_dist_area[data.area == row.area] = row.metro_dist
    
# min
    tmp = read_data.groupby('area',as_index = False).min()[['area','metro_dist']]
    data['min_metro_dist_area'] = 0
    for index,row in tmp.iterrows():
        data.min_metro_dist_area[data.area == row.area] = row.metro_dist  
# max
    tmp = read_data.groupby('area',as_index = False).max()[['area','metro_dist']]
    data['max_metro_dist_area'] = 0
    for index,row in tmp.iterrows():
        data.max_metro_dist_area[data.area == row.area] = row.metro_dist
# street
# price_room
    tmp = read_data.groupby('street_id',as_index=False).sum()[['street_id','rooms','price']]
    price_room_street = pd.DataFrame({'street_id': tmp.street_id, 'price_room_street':tmp.price/tmp.rooms } ,index = np.arange(0,tmp.price.shape[0])) 
    data['price_room_street'] = 0
    for index,row in price_room_street.iterrows():
        data.price_room_street[data.street_id == row.street_id] = row.price_room_street
    data.price_room_street = (data.price_room_street - data.price_room_street.mean())/data.price_room_street.std()
# фото
# количество
    tmp = read_data.groupby('street_id',as_index = False).sum()[['street_id','n_photos']]
    data['sum_photos_street_id'] = 0
    for index,row in tmp.iterrows():
        data.sum_photos_street_id[data.street_id == row.street_id] = row.n_photos
# mean
    tmp = read_data.groupby('street_id',as_index = False).mean()[['street_id','n_photos']]
    data['mean_photos_street_id'] = 0
    for index,row in tmp.iterrows():
        data.mean_photos_street_id[data.street_id == row.street_id] = row.n_photos
    
# std
    tmp = read_data.groupby('street_id',as_index = False).std()[['street_id','n_photos']]
    data['std_photos_street_id'] = 0
    for index,row in tmp.iterrows():
        data.std_photos_street_id[data.street_id == row.street_id] = row.n_photos
    
# min
    tmp = read_data.groupby('street_id',as_index = False).min()[['street_id','n_photos']]
    data['min_photos_street_id'] = 0
    for index,row in tmp.iterrows():
        data.min_photos_street_id[data.street_id == row.street_id] = row.n_photos  
# max
    tmp = read_data.groupby('street_id',as_index = False).max()[['street_id','n_photos']]
    data['max_photos_street_id'] = 0
    for index,row in tmp.iterrows():
        data.max_photos_street_id[data.street_id == row.street_id] = row.n_photos
# metro_dist
# количество
    tmp = read_data.groupby('street_id',as_index = False).sum()[['street_id','metro_dist']]
    data['sum_metro_dist_street_id'] = 0
    for index,row in tmp.iterrows():
        data.sum_metro_dist_street_id[data.street_id == row.street_id] = row.metro_dist
# mean
    tmp = read_data.groupby('street_id',as_index = False).mean()[['street_id','metro_dist']]
    data['mean_metro_dist_street_id'] = 0
    for index,row in tmp.iterrows():
        data.mean_metro_dist_street_id[data.street_id == row.street_id] = row.metro_dist
    
# std
    tmp = read_data.groupby('street_id',as_index = False).std()[['street_id','metro_dist']]
    data['std_metro_dist_street_id'] = 0
    for index,row in tmp.iterrows():
        data.std_metro_dist_street_id[data.street_id == row.street_id] = row.metro_dist
    
# min
    tmp = read_data.groupby('street_id',as_index = False).min()[['street_id','metro_dist']]
    data['min_metro_dist_street_id'] = 0
    for index,row in tmp.iterrows():
        data.min_metro_dist_street_id[data.street_id == row.street_id] = row.metro_dist  
# max
    tmp = read_data.groupby('street_id',as_index = False).max()[['street_id','metro_dist']]
    data['max_metro_dist_street_id'] = 0
    for index,row in tmp.iterrows():
        data.max_metro_dist_street_id[data.street_id == row.street_id] = row.metro_dist
# floor
    new_data = read_data.copy()
    new_data['price_room'] = read_data.price/read_data.rooms
    data['mean_floor_area'] = 0
    data['max_floor_area'] = 0
    data['std_floor_area'] = 0
    data['min_floor_area'] = 0
    tmp = new_data.groupby('floor', as_index=False).mean()[['floor','price_room']]
    for index,row in tmp.iterrows():
        data.mean_floor_area[data.floor == row.floor] = row.price_room
    tmp = new_data.groupby('floor',as_index=False)[['floor','price_room']].std()
    for index,row in tmp.iterrows():
        data.std_floor_area[data.floor == row.floor] = row.price_room
    tmp = new_data.groupby('floor',as_index=False).max()[['floor','price_room']]
    for index,row in tmp.iterrows():
        data.max_floor_area[data.floor == row.floor] = row.price_room
    tmp = new_data.groupby('floor',as_index=False).min()[['floor','price_room']]
    for index,row in tmp.iterrows():
        data.min_floor_area[data.floor == row.floor] = row.price_room


In [10]:
# model 3
read_data_3 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Train.csv')
new_fitch_3(read_data_3,read_data)
X_3 = read_data_3.drop(['date', 'id', 'price'],axis = 1)
y_3 = read_data_3.price
xgb_reg_3 = xgb.XGBRegressor(**parameters_xgb_reg)
xgb_reg_3.fit(X_3,y_3)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: Setti

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=3, missing=None, n_estimators=500,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [11]:
# model 4 
train = pd.read_csv('/kaggle/input/mai-ml-hw-models/Train.csv')
test = pd.read_csv('/kaggle/input/mai-ml-hw-models/Test.csv')
l_mean_price_area = dict()
for i in train.area.unique():
    l_mean_price_area[i] = int(train[train.area == i].price.mean())
l_mean_price_area[106] = int(train[train.street_id == 436].price.mean())
l_mean_price_area[214] = int(train[train.street_id == 325].price.mean())
l_mean_price_area[172] = int((3361406 + 4680566 + 5140205 + 4690463)/4)
l_mean_price_rooms = dict()
for i in train.rooms.unique():
    l_mean_price_rooms[i] = int(train[train.rooms == i].price.mean())
    l_mean_price_floor = dict()
for i in train.floor.unique():
    l_mean_price_floor[i] = int(train[train.floor == i].price.mean())
price = train.price
train = train.drop('price', axis=1)
df = train.append(test, ignore_index=True, sort=False)
p_area = []
for i in range(200_000):
    p_area.append(l_mean_price_area[df.area[i]])
df['p_area'] = p_area
p_rooms = []
for i in range(200_000):
    p_rooms.append(l_mean_price_rooms[df.rooms[i]])
df['p_rooms'] = p_rooms
p_floor = []
for i in range(200_000):
    p_floor.append(l_mean_price_floor[df.floor[i]])
df['p_floor'] = p_floor
df.build_tech[df.build_tech.isna()] = 1
df.metro_dist[df.metro_dist.isna()] = int(np.mean(df.metro_dist))
df.g_lift[df.g_lift.isna()] = 1
l = []
for i in range(200_000):
    l.append(int(str(df['street_id'][i]) + str(df['area'][i])))
df['s_a'] = l
l = df.date.unique()
n = 1
for i in l:
    df.date[df.date == i] = n
    n = n +1
df.date = df.date.astype(int)
train = df[df.id < 100_000]
X_4_test = df[df.id >= 100_000]
train = train.drop('id', axis=1)
answer_id = X_4_test.id
X_4_test = X_4_test.drop('id', axis=1)
xgb_reg_4 = xgb.XGBRegressor(colsample_bytree=0.6,learning_rate=0.08, max_depth=7, n_estimators=2000, reg_alpha=0.75, reg_lambda=0.45, subsample=0.75, seed=42) 
xgb_reg_4.fit(train, price)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be s

[18:05:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.75, verbosity=1)

In [12]:
# test_data
test_data = pd.read_csv('/kaggle/input/mai-ml-hw-models/Test.csv')

In [13]:
test_data_1 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Test.csv')
test_data_2 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Test.csv')
test_data_3 = pd.read_csv('/kaggle/input/mai-ml-hw-models/Test.csv')

In [14]:
X_1_test = test_data_1.drop(['date', 'id'],axis = 1)

In [15]:
new_fitch_2(test_data_2,read_data)
X_2_test=test_data_2.drop(['date', 'id'],axis = 1)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: Setti

In [16]:
new_fitch_3(test_data_3,read_data)
X_3_test=test_data_3.drop(['date', 'id'],axis = 1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: Setti

In [17]:
preds = (xgb_reg_1.predict(X_1_test) + xgb_reg_2.predict(X_2_test)+xgb_reg_3.predict(X_3_test) + xgb_reg_4.predict(X_4_test))/4

In [18]:
answer = pd.DataFrame({'id': test_data.id , 'price':preds})
answer.to_csv('result.csv',index = False)

In [19]:
answer

,id,price
0,100000,4720921.50
1,100001,4611734.50
2,100002,2762985.25
3,100003,2188933.50
4,100004,2862984.25
...,...,...
99995,199995,6203809.00
99996,199996,6755510.00
99997,199997,3201343.50
99998,199998,3705280.75
